## Unveiling Pulsars: Machine Learning for Pulsar Identification?

#### Introduction

The HTRU2 dataset comes from a project that looks for rare stars called pulsars, which send radio signals to Earth. These stars are essential for understanding space and the relation between stars.

Pulsars spin quickly, and as they do, they send out a repeating radio signal that can be detected using radio telescopes. This signal is like a specific code for each pulsar, but it's hard to find because there's a lot of interference and noise.

The dataset we'll use includes measurements like the average pulse, its variability, shape, and other features. It has a total of 17,898 examples. Among these, 1,639 are positive (real pulsars), and 16,259 are negative (noise). The data comes in two formats, CSV and ARFF, and each entry has a set of values for these measurements, with 0 indicating noise and 1 indicating a real pulsar.

To discover hidden pulsars, we use Python that learns from examples. We tell the computer what real pulsars look like and what noise looks like. Then, it tries to sort new examples into "pulsar" or "not pulsar." Our goal is to create a program that can tell the difference between pulsars and noise, which will help us discover more about these incredible stars.


#### Preliminary exploratory data analysis

- Read in dataset
- clean and wrangle
- summarize table in one data
- visualize data with one plot relevant to our analysis

In [ ]:
# import dataset

In [ ]:
# clean

In [ ]:
# table - using only training data

In [ ]:
# plot - using only training data

#### Methods

Explain

Describe

#### Expected outcomes and Significance

- What we will find? Look at visualizations, data, etc.
- Impact of findings?
- Future questions? 